In [1]:
#include <algoviz/SVG.hpp>
#include <iostream>
#include <stdlib.h>
#include <vector>
#include "Obstacle.hpp"
#include "Bird.hpp"
#include <cmath>

In [2]:
void runGame(SVG &drawing, Bird &bird, vector<Obstacle*> oblist) {
   
    string key;
    bool collision = false;
    int highscore = 0;
    
    // Wait for Enter Key to begin game
    while (key != "Enter") {
        key = drawing.waitForKey();
    }
    
    // Setup bird 
    bird.center();
    bird.setNormalSize();
    
    // Remove obsolete obstacles
    for ( auto it = oblist.begin(); it != oblist.end(); it++ ) {
        (*it)->remove();
        oblist.erase(it--);
    }
    
    Obstacle *obst = new Obstacle(drawing, rand() % 600);
    oblist.push_back(obst);
    
    // Main Loop
    while (!collision) {

    drawing.setStatusMsg("Highscore: " + to_string(highscore));
       key = drawing.lastKey();

        if (key == "ArrowUp") {
            bird.moveUp();
            key = "";
        } else {
            bird.moveDown();
        }
        
        // Move all obstacles in the list
        for (auto it = oblist.begin(); it != oblist.end(); it++) {
            (*it)->move();
        }
        
        // Create new obstacle  
        if ((*oblist.back()).getX() < 600) {
            Obstacle *o = new Obstacle(drawing, rand() % 600);
            oblist.push_back(o);
        }

        // Delete obstacles
        if ((*oblist.front()).getX() < -40) {
            Obstacle *o = oblist.front();
            oblist.erase(oblist.begin());
            o->remove();
            delete o;
        }  

        // Check for collision with top and bottom
        if (bird.getY() + 40 > 1000 || bird.getY() - 40 < 0) {
            collision = true;
        }
        
        // Check for collision with obstacles
        for (Obstacle *o : oblist) {
            if (abs(bird.getX() + 20 - o->getX()) < 20 || abs(bird.getX() - 20 - o->getX()) < 20) {
                if (bird.getY() - 30 < o->getUpperHeight() || bird.getY() + 45 > o->getLowerY()) {
                    collision = true;
                }
            } 
        }
        
        // Check if bee passes an obstacle
        for (Obstacle *o : oblist) {
            if (bird.getX() + 20 > o->getX() + 50 && o->getPassed() == false) {
                highscore++;
                o->setPassed();
            }
        }

        // Small wait timer for smoother animations
        AlgoViz::sleep(5);
       
    }
    
    drawing.setStatusMsg("Highscore: " + to_string(highscore) + " Bee is over! Hit enter to play again");
    runGame(drawing, bird, oblist);
}

In [ ]:
AlgoViz::clear();

// Declare global variables, setup drawing
SVG drawing = SVG(960, 1000, "Flappy Bee");
Image bg = Image("img/BG.svg", 480, 500, 1000, 1000, &drawing);
drawing.setStatusMsg("I want to fly and enjoy the sun! Press Enter!");
Bird bird = Bird(drawing);
vector<Obstacle*> oblist;
srand(time(NULL));

runGame(drawing, bird, oblist);



